In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain_ai21 import ChatAI21
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2


In [3]:
load_dotenv()

True

In [4]:
os.environ["AI21_API_KEY"]= os.getenv("AI21_API_KEY")

In [5]:
# llm = ChatOpenAI(openai_api_key=os.getenv("OPENAI_API_KEY"), model_name="gpt-3.5-turbo", temperature=0.5)
llm = ChatAI21(model="jamba-instruct")
llm

ChatAI21(client=<ai21.clients.studio.ai21_client.AI21Client object at 0x7a05c81150a0>, api_key=SecretStr('**********'), api_host='https://api.ai21.com', timeout_sec=300.0, model='jamba-instruct', _chat_adapter=<langchain_ai21.chat.chat_adapter.JambaChatCompletionsAdapter object at 0x7a05c81153a0>)

In [6]:
response_json = {
    "1" : {
        "mcq" : "multiple choice question",
        "options" : {
            "a" : "choice here",
            "b" : "choice here",
            "c" : "choice here",
            "d" : "choice here",
        },
        "correct" : "correct answer",
    } ,
    "2" : {
        "mcq" : "multiple choice question",
        "options" : {
            "a" : "choice here",
            "b" : "choice here",
            "c" : "choice here",
            "d" : "choice here",
        },
        "correct" : "correct answer",
    } ,
    "3" : {
        "mcq" : "multiple choice question",
        "options" : {
            "a" : "choice here",
            "b" : "choice here",
            "c" : "choice here",
            "d" : "choice here",
        },
        "correct" : "correct answer",
    } ,
}

In [7]:
template = """
Text : {text}
You are an expert MCQ maker. Given the above, it is your job to create a quiz of {number} multiple choice \
questions for {subject} students in {tone} tone. Make sure the questions are not repeated and check all \
the questions to be confirming the text as well. Give your response in a valid format like RESPONSE_JSON below \
and use it as a guide, \
Ensure to make {number} MCQs
RESPONSE_JSON :
{response_json}
"""

In [8]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=template
)

In [9]:
quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

/home/sanskar/anaconda3/envs/mcqgenvenv/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [10]:
template2 = """
You are an expert English Grammarian and Writer. Given a multiple choice quiz for {subject} students. \
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use \
at max 50 words for the compelxity. If the quiz is not as per the cognitive and analytical abilities of \
the students, update the quiz questions which needs to be changed and change the tone such that it perfectly \
fits the students. \
Quiz MCQs:
{quiz}

Check the above quiz as an expert english writer.
"""

In [11]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject", "quiz"], template=template2)

In [12]:
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [13]:
chain = SequentialChain(chains=[quiz_chain,review_chain], 
                        input_variables=["text", "number", "subject", "tone", "response_json"],
                        output_variables=["quiz", "review"], verbose=True)

In [14]:
with open("../data.txt", 'r') as f:
    text = f.read()

In [15]:
text

"Machine learning classification algorithms are used to assign labels to instances based on their features. Different algorithms are suitable for different types of data and problems. Here is a comparison of various commonly used classification algorithms, including when to use each and a brief detail about each one.\n\n1. Logistic Regression\nWhen to Use:\n\nBinary classification problems.\nWhen the relationship between the features and the target variable is approximately linear.\nWhen interpretability is important.\nDetails:\nLogistic regression models the probability that a given instance belongs to a particular class. It uses the logistic function to squeeze the output of a linear equation between 0 and 1. It is simple, efficient, and provides probabilistic interpretations.\n\n2. Decision Trees\nWhen to Use:\n\nWhen interpretability is important.\nWhen you need a non-linear model.\nWhen dealing with both numerical and categorical data.\nDetails:\nDecision trees split the data into

In [16]:
response_json = json.dumps(response_json)

In [17]:
with get_openai_callback() as cb:
    response = chain(
        {
            "text": text,
            "number": 5,
            "subject": "Machine Learning",
            "tone" : "Intermediate",
            "response_json" : response_json
        }
    )

/home/sanskar/anaconda3/envs/mcqgenvenv/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text : Machine learning classification algorithms are used to assign labels to instances based on their features. Different algorithms are suitable for different types of data and problems. Here is a comparison of various commonly used classification algorithms, including when to use each and a brief detail about each one.

1. Logistic Regression
When to Use:

Binary classification problems.
When the relationship between the features and the target variable is approximately linear.
When interpretability is important.
Details:
Logistic regression models the probability that a given instance belongs to a particular class. It uses the logistic function to squeeze the output of a linear equation between 0 and 1. It is simple, efficient, and provides probabilistic interpretations.

2. Decision Trees
When to Use:

When interpretability is important.
When you need a non-linear model.
When de

In [18]:
response

{'text': "Machine learning classification algorithms are used to assign labels to instances based on their features. Different algorithms are suitable for different types of data and problems. Here is a comparison of various commonly used classification algorithms, including when to use each and a brief detail about each one.\n\n1. Logistic Regression\nWhen to Use:\n\nBinary classification problems.\nWhen the relationship between the features and the target variable is approximately linear.\nWhen interpretability is important.\nDetails:\nLogistic regression models the probability that a given instance belongs to a particular class. It uses the logistic function to squeeze the output of a linear equation between 0 and 1. It is simple, efficient, and provides probabilistic interpretations.\n\n2. Decision Trees\nWhen to Use:\n\nWhen interpretability is important.\nWhen you need a non-linear model.\nWhen dealing with both numerical and categorical data.\nDetails:\nDecision trees split the 

In [23]:
quiz = json.loads(response.get("quiz"))
print(quiz)

{'1': {'mcq': 'Which classification algorithm is suitable for binary classification problems and when the relationship between the features and the target variable is approximately linear?', 'options': {'a': 'Decision Trees', 'b': 'Logistic Regression', 'c': 'Naive Bayes', 'd': 'Random Forest'}, 'correct': 'b'}, '2': {'mcq': 'Which classification algorithm is known for its interpretability and can handle both numerical and categorical data?', 'options': {'a': 'Support Vector Machines (SVM)', 'b': 'K-Nearest Neighbors (KNN)', 'c': 'Decision Trees', 'd': 'Naive Bayes'}, 'correct': 'c'}, '3': {'mcq': 'What is the key advantage of using Random Forest over Decision Trees?', 'options': {'a': 'Random Forest is more interpretable', 'b': 'Random Forest handles missing values well', 'c': 'Random Forest is less prone to overfitting', 'd': 'Random Forest is faster to train'}, 'correct': 'c'}, '4': {'mcq': "Which classification algorithm is based on Bayes' theorem and assumes that features are inde

In [20]:
review = response.get("review")
print(review)

The quiz is at an appropriate level of complexity for Machine Learning students. However, a minor error in question 5 needs to be corrected. Instead of "high-dimensional spaces," it should be "high-dimensional datasets" or "high-dimensional feature spaces." Additionally, the tone of the questions can be slightly adjusted for clarity and conciseness. Here's the updated version:

{"1": {"mcq": "Which algorithm is best for binary classification with a linear relationship between features and targets?",
"options": {"a": "Decision Trees", "b": "Logistic Regression", "c": "Naive Bayes", "d": "Random Forest"}, "correct": "b"},
"2": {"mcq": "Which classification algorithm is most interpretable and handles both numerical and categorical data?",
"options": {"a": "Support Vector Machines (SVM)", "b": "K-Nearest Neighbors (KNN)", "c": "Decision Trees", "d": "Naive Bayes"}, "correct": "c"},
"3": {"mcq": "What is the main advantage of Random Forest over Decision Trees?",
"options": {"a": "Random For

In [26]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join([f"{option} : {option_value}" for option, option_value in value["options"].items()])
    correct = value["correct"]
    quiz_table_data.append({"MCQ":mcq, "Choices":options, "Correct":correct})

In [28]:
quiz_table_data

[{'MCQ': 'Which classification algorithm is suitable for binary classification problems and when the relationship between the features and the target variable is approximately linear?',
  'Choices': 'a : Decision Trees | b : Logistic Regression | c : Naive Bayes | d : Random Forest',
  'Correct': 'b'},
 {'MCQ': 'Which classification algorithm is known for its interpretability and can handle both numerical and categorical data?',
  'Choices': 'a : Support Vector Machines (SVM) | b : K-Nearest Neighbors (KNN) | c : Decision Trees | d : Naive Bayes',
  'Correct': 'c'},
 {'MCQ': 'What is the key advantage of using Random Forest over Decision Trees?',
  'Choices': 'a : Random Forest is more interpretable | b : Random Forest handles missing values well | c : Random Forest is less prone to overfitting | d : Random Forest is faster to train',
  'Correct': 'c'},
 {'MCQ': "Which classification algorithm is based on Bayes' theorem and assumes that features are independent given the class label?",

In [29]:
quiz = pd.DataFrame(quiz_table_data)

In [30]:
quiz

,MCQ,Choices,Correct
0,Which classification algorithm is suitable for...,a : Decision Trees | b : Logistic Regression |...,b
1,Which classification algorithm is known for it...,a : Support Vector Machines (SVM) | b : K-Near...,c
2,What is the key advantage of using Random Fore...,a : Random Forest is more interpretable | b : ...,c
3,Which classification algorithm is based on Bay...,a : Support Vector Machines (SVM) | b : K-Near...,c
4,What is the main disadvantage of using Neural ...,a : They are not suitable for high-dimensional...,d
